이 자료는 위키독스 딥 러닝을 이용한 자연어 처리 입문의 코사인 유사도 튜토리얼 자료입니다.  

링크 : https://wikidocs.net/24603

# 1. 코사인 유사도(Cosine Similarity)

##### 0) 라이브러리 불러오기

In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

##### 1) 함수 정의

In [2]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

##### 2) 데이터 불러오기

In [3]:
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

##### 3) 출력

In [4]:
print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :',cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


# 2. 추천 시스템 제작 by 유사도

##### 0) 라이브러리 불러오기

In [5]:
# (1) 기본 라이브러리
import pandas as pd

# (2) 딥러닝 라이브러리
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

##### 1) 데이터 불러오기

In [6]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)

In [7]:
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


##### 2) 데이터 전처리

(1) 일부 데이터만 불러오기

In [8]:
data = data.head(20000)

(2) 결측치 제거

In [9]:
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())

overview 열의 결측값의 수: 135


In [10]:
data['overview'] = data['overview'].fillna('')

##### 3) 결과 추출

(1) TF-IDF 추출

In [11]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])

In [12]:
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (20000, 47487)


(2) Cosine 유사도 추출

In [13]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [14]:
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [15]:
cosine_sim

array([[1.        , 0.01575748, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04907345, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.08375766],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08375766, 0.        ,
        1.        ]])

##### 4) Title -> Index -> Cosine Simularity

In [16]:
title_to_index = dict(zip(data['title'], data.index)) # !!!

In [17]:
idx = title_to_index['Father of the Bride Part II'] # !!!

4


In [ ]:
idx

In [22]:
def get_recommendations(title, cosine_sim=cosine_sim): # !!!
    # (1) 영화 title의 index
    idx = title_to_index[title]

    # (2) 영화 index의 Cosine Simularity 뭉탱이
    sim_scores = list(enumerate(cosine_sim[idx]))
    print(f'sim_scores : {sim_scores}')

    # (2) 영화 index의 Cosine Simularity 내림차순
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    print(f'sim_scores : {sim_scores}')

    # (3) 영화 index의 Cosine Simularity 상위 10개 (본인 제외)
    sim_scores = sim_scores[1:11]
    print(f'sim_scores : {sim_scores}')

    # (4) 영화 index의 Cosine Simularity 상위 10개 index
    movie_indices = [idx[0] for idx in sim_scores]
    print(f'movie_indices : {movie_indices}')

    # (5) 영화 index의 Cosine Simularity 상위 10개 title
    return data['title'].iloc[movie_indices]

In [23]:
get_recommendations('The Dark Knight Rises')

sim_scores : [(0, 0.0), (1, 0.006160419774722816), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.03173549559596929), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.029344086009260913), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.05443116495006318), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.01094038722222377), (24, 0.01041175762459986), (25, 0.0), (26, 0.0), (27, 0.014071662980067425), (28, 0.0), (29, 0.0), (30, 0.01539309229750991), (31, 0.007664043345976653), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.009078812229763538), (36, 0.010383157259466633), (37, 0.0), (38, 0.0037551246122038404), (39, 0.03553315588396042), (40, 0.0), (41, 0.0), (42, 0.0022850613717058307), (43, 0.0), (44, 0.020037489465932312), (45, 0.0), (46, 0.03411526202311631), (47, 0.005604843383253618), (48, 0.0), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.012538091512842154), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (6

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object